# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews
import cartopy

# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("CSV/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temperature
0,0,cherskiy,68.7500,161.3000,245.99,95,100,1.14,RU,1681238402,24.599
1,1,bluff,-46.6000,168.3333,284.71,92,100,8.46,NZ,1681238405,28.471
2,2,puerto ayora,-0.7393,-90.3518,304.13,83,80,2.68,EC,1681238406,30.413
3,3,punta arenas,-53.1500,-70.9167,281.16,65,75,8.75,CL,1681238406,28.116
4,4,vaini,-21.2000,-175.2000,298.24,94,20,2.57,TO,1681238407,29.824


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Configure the map plot
city_map_plot=city_data_df.hvplot.points("Lng","Lat", 
                                      color= "City", geo=True,
                                         frame_width=700,
                                         frame_height=500,
                                         size="Humidity",
                                      scale=1, alpha=0.5,
                                         tiles="OSM")

# Display the map
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values

#Temperature
city_data_df = city_data_df.loc[(city_data_df["Max Temperature"] < 27) & (city_data_df["Max Temperature"] > 21)]

#Wind
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] ==0)]

#Drop Nulls 
city_data_df = city_data_df.dropna()

#Display data 
city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temperature
134,134,anadyr,64.7500,177.4833,249.43,70,0,1.00,RU,1681238502,24.943
191,191,qaanaaq,77.4840,-69.3632,252.95,82,0,0.41,GL,1681238521,25.295
226,226,nome,64.5011,-165.4064,249.91,77,0,0.00,US,1681238538,24.991
250,250,barrow,71.2906,-156.7887,244.16,76,0,3.09,US,1681238553,24.416


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=city_data_df.copy()
del hotel_df["Date"]
del hotel_df["Wind Speed"]
del hotel_df["Cloudiness"]
del hotel_df["City_ID"]
del hotel_df["Max Temperature"]
del hotel_df["Max Temp"]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
134,anadyr,64.7500,177.4833,70,RU,
191,qaanaaq,77.4840,-69.3632,82,GL,
226,nome,64.5011,-165.4064,77,US,
250,barrow,71.2906,-156.7887,76,US,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [34]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "format":"json",
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}" 
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
anadyr - nearest hotel: Гостевой дом
qaanaaq - nearest hotel: Hotel Qaanaaq
nome - nearest hotel: No hotel found
barrow - nearest hotel: No hotel found


,City,Lat,Lng,Humidity,Country,Hotel Name
134,anadyr,64.7500,177.4833,70,RU,Гостевой дом
191,qaanaaq,77.4840,-69.3632,82,GL,Hotel Qaanaaq
226,nome,64.5011,-165.4064,77,US,No hotel found
250,barrow,71.2906,-156.7887,76,US,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

hotel_map=hotel_df.hvplot.points("Lng","Lat",  tiles="OSM", 
                                      geo=True, 
                                      color= "City",scale=1,
                                      alpha=0.5, 
                                      frame_height=500,
                                      frame_width=700, 
                                      size="Humidity", hover_cols=["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)